# L3 M9.4: Human-in-the-Loop for High-Stakes Decisions

**Track:** Finance AI (Domain-Specific)  
**Module:** M9 - Financial Compliance & Risk  
**Duration:** 40-45 minutes  
**Level:** L2 SkillElevate

---

## Learning Arc

This notebook implements a **collaborative decision-making model** where AI processes massive datasets while humans provide contextual judgment and accountability.

**You will learn:**

1. **Risk Classification** - Implementing risk-based query classification for routing
2. **Expertise Routing** - Matching queries to domain specialists with appropriate SLAs
3. **Audit Trail Creation** - Building tamper-proof logs with cryptographic hash chains
4. **Approval Workflows** - Role-based access control with escalation hierarchies
5. **SLA Compliance** - Time-aware workflows balancing thoroughness with market timing
6. **Escalation Mechanisms** - Structured paths for urgent/complex decisions
7. **Real-World Application** - Tesla $8M position increase case study
8. **Failure Analysis** - Common pitfalls and debugging strategies

**By the end:**
- Understand the five HITL design principles (risk-proportional review, expertise-matched routing, time-aware workflows, audit-ready by default, fail-safe defaults)
- Implement production-ready workflow orchestration
- Handle regulatory compliance requirements (SOX, FINRA, SEC, ECOA)
- Prevent catastrophic financial errors while maintaining analytical speed

In [ ]:
# OFFLINE Mode Guard and Setup
import os
import sys
from pathlib import Path

# Set up path to import from src/
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))

# Check if external services are enabled
OPENAI_ENABLED = os.getenv("OPENAI_ENABLED", "false").lower() == "true"

if not OPENAI_ENABLED:
    print("⚠️ OPENAI disabled. Core workflow operates independently.")
    print("To enable RAG integration: Set OPENAI_ENABLED=true in .env")
else:
    print("✅ OPENAI enabled for RAG integration")

print("\n" + "="*60)
print("L3 M9.4: Human-in-the-Loop for High-Stakes Decisions")
print("="*60)

## Section 1: Risk Classification Framework

**Concept:** Risk-proportional review matches scrutiny depth to financial risk.

**Classification Criteria:**
- **CRITICAL:** MNPI (material non-public information) or $10M+ transactions
- **HIGH:** $1M+ transactions, investment decisions, portfolio rebalancing, credit approvals
- **MEDIUM:** $100K-1M transactions, low model confidence (<70%), earnings analysis
- **LOW:** Informational queries, historical data, definitions

**Expected:** SAVED_SECTION:1

In [ ]:
from src.l3_m9_financial_compliance_risk import classify_risk, RiskLevel

# Example 1: CRITICAL risk - MNPI disclosure
risk1, reason1 = classify_risk(
    query_text="Approve Q4 earnings disclosure",
    contains_mnpi=True
)
print(f"Query 1: MNPI Disclosure")
print(f"  Risk: {risk1.value}")
print(f"  Reason: {reason1}")
# Expected: CRITICAL risk due to MNPI

print("\n" + "-"*60 + "\n")

# Example 2: HIGH risk - Large transaction
risk2, reason2 = classify_risk(
    query_text="Approve $8M Tesla position increase",
    transaction_amount=8_000_000,
    action_type="investment_decision"
)
print(f"Query 2: $8M Tesla Position")
print(f"  Risk: {risk2.value}")
print(f"  Reason: {reason2}")
# Expected: HIGH risk, >$1M threshold

print("\n" + "-"*60 + "\n")

# Example 3: MEDIUM risk - Low model confidence
risk3, reason3 = classify_risk(
    query_text="Analyze Q3 earnings for uncertainty",
    model_confidence=0.65
)
print(f"Query 3: Low Confidence Analysis")
print(f"  Risk: {risk3.value}")
print(f"  Reason: {reason3}")
# Expected: MEDIUM risk, <70% confidence

print("\n" + "-"*60 + "\n")

# Example 4: LOW risk - Informational
risk4, reason4 = classify_risk(
    query_text="What is Tesla's market cap?",
    transaction_amount=0,
    action_type="informational"
)
print(f"Query 4: Market Cap Lookup")
print(f"  Risk: {risk4.value}")
print(f"  Reason: {reason4}")
# Expected: LOW risk, informational only

print("\n✅ SAVED_SECTION:1")

## Section 2: Expertise-Matched Routing

**Concept:** Route queries to domain specialists based on risk level.

**Routing Hierarchy:**
- **CFO:** CRITICAL risk, 2-hour SLA, committee approval required
- **Senior Analyst:** HIGH risk, 4-hour SLA, second opinion for $10M+
- **Analyst:** MEDIUM risk, 8-hour SLA
- **Junior Analyst:** LOW risk, 24-hour SLA

**Expected:** SAVED_SECTION:2

In [ ]:
from src.l3_m9_financial_compliance_risk import route_to_reviewer

# Route CRITICAL risk query
routing_critical = route_to_reviewer(RiskLevel.CRITICAL)
print("CRITICAL Risk Routing:")
print(f"  Reviewer: {routing_critical['reviewer_role']}")
print(f"  SLA: {routing_critical['sla_hours']} hours")
print(f"  Second Opinion: {routing_critical['requires_second_opinion']}")
print(f"  Committee: {routing_critical['approval_committee_needed']}")
# Expected: CFO, 2h SLA, second opinion + committee

print("\n" + "-"*60 + "\n")

# Route HIGH risk $10M+ transaction
routing_high_large = route_to_reviewer(RiskLevel.HIGH, transaction_amount=12_000_000)
print("HIGH Risk ($10M+) Routing:")
print(f"  Reviewer: {routing_high_large['reviewer_role']}")
print(f"  SLA: {routing_high_large['sla_hours']} hours")
print(f"  Second Opinion: {routing_high_large['requires_second_opinion']}")
# Expected: Senior Analyst, 4h SLA, second opinion required

print("\n" + "-"*60 + "\n")

# Route standard HIGH risk
routing_high = route_to_reviewer(RiskLevel.HIGH, transaction_amount=5_000_000)
print("HIGH Risk (Standard) Routing:")
print(f"  Reviewer: {routing_high['reviewer_role']}")
print(f"  SLA: {routing_high['sla_hours']} hours")
print(f"  Second Opinion: {routing_high['requires_second_opinion']}")
# Expected: Senior Analyst, 4h SLA, no second opinion

print("\n" + "-"*60 + "\n")

# Route MEDIUM risk
routing_medium = route_to_reviewer(RiskLevel.MEDIUM)
print("MEDIUM Risk Routing:")
print(f"  Reviewer: {routing_medium['reviewer_role']}")
print(f"  SLA: {routing_medium['sla_hours']} hours")
# Expected: Analyst, 8h SLA

print("\n" + "-"*60 + "\n")

# Route LOW risk
routing_low = route_to_reviewer(RiskLevel.LOW)
print("LOW Risk Routing:")
print(f"  Reviewer: {routing_low['reviewer_role']}")
print(f"  SLA: {routing_low['sla_hours']} hours")
# Expected: Junior Analyst, 24h SLA

print("\n✅ SAVED_SECTION:2")

## Section 3: Audit-Ready by Default

**Concept:** Tamper-proof audit trails with cryptographic hash chains.

**Audit Entry Components:**
- Unique audit_id
- Timestamp (ISO 8601 UTC)
- User metadata (user_id, query_text, rag_response)
- Review data (reviewer_id, decision_outcome, supporting_evidence)
- Hash chain (previous_hash, current_hash)

**SOX Compliance:** 7-year retention with tamper detection.

**Expected:** SAVED_SECTION:3

In [ ]:
from src.l3_m9_financial_compliance_risk import (
    create_audit_entry,
    verify_hash_chain,
    DecisionOutcome
)

# Create audit entry 1
entry1 = create_audit_entry(
    user_id="pm_001",
    query_text="Approve $8M Tesla position increase",
    rag_response={"recommendation": "Increase", "confidence": 0.85},
    risk_classification=RiskLevel.HIGH
)

print("Audit Entry 1 Created:")
print(f"  Audit ID: {entry1['audit_id']}")
print(f"  Timestamp: {entry1['timestamp']}")
print(f"  Hash: {entry1['current_hash'][:32]}...")
# Expected: Valid audit entry with hash

print("\n" + "-"*60 + "\n")

# Create chained entry 2
entry2 = create_audit_entry(
    user_id="pm_002",
    query_text="What is market cap?",
    rag_response={"answer": "Market capitalization..."},
    risk_classification=RiskLevel.LOW,
    previous_hash=entry1["current_hash"]
)

print("Audit Entry 2 (Chained):")
print(f"  Audit ID: {entry2['audit_id']}")
print(f"  Previous Hash: {entry2['previous_hash'][:32]}...")
print(f"  Current Hash: {entry2['current_hash'][:32]}...")
print(f"  Chain Valid: {entry2['previous_hash'] == entry1['current_hash']}")
# Expected: Valid chain linkage

print("\n" + "-"*60 + "\n")

# Verify hash chain
audit_trail = [entry1, entry2]
is_valid, error = verify_hash_chain(audit_trail)

print("Hash Chain Verification:")
print(f"  Valid: {is_valid}")
print(f"  Error: {error if error else 'None'}")
# Expected: Valid chain, no errors

print("\n" + "-"*60 + "\n")

# Demonstrate tampering detection
tampered_entry = entry1.copy()
tampered_entry['query_text'] = "TAMPERED QUERY"
tampered_trail = [tampered_entry, entry2]
is_valid_tampered, error_tampered = verify_hash_chain(tampered_trail)

print("Tampered Chain Verification:")
print(f"  Valid: {is_valid_tampered}")
print(f"  Error: {error_tampered}")
# Expected: Invalid chain, hash mismatch detected

print("\n✅ SAVED_SECTION:3")

## Section 4: Role-Based Access Control

**Concept:** Hierarchical approval chains ensure only authorized personnel approve decisions.

**Role Hierarchy (Level 1-5):**
1. Junior Analyst
2. Analyst
3. Senior Analyst
4. Head of Trading
5. CFO

**Validation:** Reviewer must meet minimum role level; second opinion required for critical decisions.

**Expected:** SAVED_SECTION:4

In [ ]:
from src.l3_m9_financial_compliance_risk import validate_approval_chain

# Valid approval: Senior Analyst for HIGH risk
is_valid1, reason1 = validate_approval_chain(
    decision_outcome=DecisionOutcome.APPROVED,
    reviewer_role="Senior Analyst",
    required_role="Senior Analyst"
)
print("Scenario 1: Senior Analyst Approval")
print(f"  Valid: {is_valid1}")
print(f"  Reason: {reason1}")
# Expected: Valid, meets requirements

print("\n" + "-"*60 + "\n")

# Invalid: Insufficient role
is_valid2, reason2 = validate_approval_chain(
    decision_outcome=DecisionOutcome.APPROVED,
    reviewer_role="Analyst",
    required_role="Senior Analyst"
)
print("Scenario 2: Insufficient Role")
print(f"  Valid: {is_valid2}")
print(f"  Reason: {reason2}")
# Expected: Invalid, insufficient role level

print("\n" + "-"*60 + "\n")

# Missing required second opinion
is_valid3, reason3 = validate_approval_chain(
    decision_outcome=DecisionOutcome.APPROVED,
    reviewer_role="Senior Analyst",
    required_role="Senior Analyst",
    requires_second_opinion=True,
    second_reviewer_role=None
)
print("Scenario 3: Missing Second Opinion")
print(f"  Valid: {is_valid3}")
print(f"  Reason: {reason3}")
# Expected: Invalid, second opinion required

print("\n" + "-"*60 + "\n")

# Valid with second opinion
is_valid4, reason4 = validate_approval_chain(
    decision_outcome=DecisionOutcome.APPROVED,
    reviewer_role="Senior Analyst",
    required_role="Senior Analyst",
    requires_second_opinion=True,
    second_reviewer_role="Senior Analyst"
)
print("Scenario 4: Valid Second Opinion")
print(f"  Valid: {is_valid4}")
print(f"  Reason: {reason4}")
# Expected: Valid, all requirements met

print("\n✅ SAVED_SECTION:4")

## Section 5: Time-Aware Workflows (SLA Compliance)

**Concept:** Balance thoroughness with market timing through SLA tracking.

**SLA Tiers:**
- **Urgent (2h):** CRITICAL risk, market-moving events
- **Standard (4h):** HIGH risk, investment decisions
- **Extended (8-24h):** MEDIUM/LOW risk, analytical queries

**Monitoring:** Real-time tracking, warnings at 80% threshold, auto-escalation on breach.

**Expected:** SAVED_SECTION:5

In [ ]:
from src.l3_m9_financial_compliance_risk import check_sla_compliance
from datetime import datetime, timedelta

# Scenario 1: Within SLA
submission_time1 = datetime.utcnow() - timedelta(hours=2)
review_time1 = datetime.utcnow()

sla1 = check_sla_compliance(
    submission_time=submission_time1,
    review_time=review_time1,
    sla_hours=4
)

print("Scenario 1: Within SLA (2h of 4h)")
print(f"  Compliant: {sla1['is_compliant']}")
print(f"  Elapsed: {sla1['elapsed_hours']:.2f}h")
print(f"  Status: {sla1['status']}")
# Expected: Compliant, ~2h elapsed

print("\n" + "-"*60 + "\n")

# Scenario 2: SLA Breach
submission_time2 = datetime.utcnow() - timedelta(hours=6)
review_time2 = datetime.utcnow()

sla2 = check_sla_compliance(
    submission_time=submission_time2,
    review_time=review_time2,
    sla_hours=4
)

print("Scenario 2: SLA Breach (6h of 4h)")
print(f"  Compliant: {sla2['is_compliant']}")
print(f"  Elapsed: {sla2['elapsed_hours']:.2f}h")
print(f"  Status: {sla2['status']}")
# Expected: Non-compliant, exceeded by 2h

print("\n" + "-"*60 + "\n")

# Scenario 3: Pending with time remaining
submission_time3 = datetime.utcnow() - timedelta(hours=1)

sla3 = check_sla_compliance(
    submission_time=submission_time3,
    review_time=None,  # Still pending
    sla_hours=4
)

print("Scenario 3: Pending Review (1h elapsed of 4h)")
print(f"  Remaining: {sla3['remaining_hours']:.2f}h")
print(f"  Status: {sla3['status']}")
# Expected: ~3h remaining

print("\n" + "-"*60 + "\n")

# Scenario 4: Pending and overdue
submission_time4 = datetime.utcnow() - timedelta(hours=5)

sla4 = check_sla_compliance(
    submission_time=submission_time4,
    review_time=None,
    sla_hours=4
)

print("Scenario 4: Pending and Overdue (5h of 4h SLA)")
print(f"  Remaining: {sla4['remaining_hours']:.2f}h")
print(f"  Status: {sla4['status']}")
# Expected: Negative remaining, overdue warning

print("\n✅ SAVED_SECTION:5")

## Section 6: Escalation Hierarchies

**Concept:** Structured escalation paths for urgent/complex decisions requiring higher authority.

**Escalation Triggers:**
- SLA breach (time-based)
- Reviewer uncertainty (complexity-based)
- Conflicting second opinions (dispute-based)
- Regulatory concerns (compliance-based)

**Fail-Safe:** Unknown roles escalate to highest authority (CFO).

**Expected:** SAVED_SECTION:6

In [ ]:
from src.l3_m9_financial_compliance_risk import escalate_decision

escalation_path = ["Analyst", "Senior Analyst", "Head of Trading", "CFO"]

# Scenario 1: Normal escalation
escalation1 = escalate_decision(
    current_reviewer_role="Analyst",
    escalation_path=escalation_path,
    reason="SLA breach - 6 hours elapsed"
)

print("Scenario 1: Analyst → Senior Analyst")
print(f"  Next Reviewer: {escalation1['next_reviewer_role']}")
print(f"  Priority: {escalation1['priority']}")
print(f"  Reason: {escalation1['escalation_reason']}")
# Expected: Senior Analyst, HIGH priority

print("\n" + "-"*60 + "\n")

# Scenario 2: Multiple escalations
escalation2 = escalate_decision(
    current_reviewer_role="Senior Analyst",
    escalation_path=escalation_path,
    reason="Complex regulatory interpretation needed"
)

print("Scenario 2: Senior Analyst → Head of Trading")
print(f"  Next Reviewer: {escalation2['next_reviewer_role']}")
print(f"  Priority: {escalation2['priority']}")
# Expected: Head of Trading, HIGH priority

print("\n" + "-"*60 + "\n")

# Scenario 3: Already at highest level
escalation3 = escalate_decision(
    current_reviewer_role="CFO",
    escalation_path=escalation_path,
    reason="Requires board approval"
)

print("Scenario 3: CFO → Committee Required")
print(f"  Next Reviewer: {escalation3['next_reviewer_role']}")
print(f"  Priority: {escalation3['priority']}")
print(f"  Committee: {escalation3['requires_committee']}")
# Expected: CFO (no higher level), CRITICAL, committee needed

print("\n" + "-"*60 + "\n")

# Scenario 4: Fail-safe (unknown role)
escalation4 = escalate_decision(
    current_reviewer_role="Unknown Role",
    escalation_path=escalation_path,
    reason="System error"
)

print("Scenario 4: Unknown Role → Fail-Safe Escalation")
print(f"  Next Reviewer: {escalation4['next_reviewer_role']}")
print(f"  Priority: {escalation4['priority']}")
print(f"  Reason: {escalation4['escalation_reason']}")
# Expected: CFO (highest level), CRITICAL priority

print("\n✅ SAVED_SECTION:6")

## Section 7: Complete Workflow Demonstration

**Concept:** End-to-end workflow from query submission to review completion.

**Workflow Steps:**
1. Submit query with transaction details
2. Automatic risk classification
3. Route to appropriate reviewer
4. Create audit trail entry
5. Human review and decision
6. Update audit trail with review
7. Verify hash chain integrity

**Expected:** SAVED_SECTION:7

In [ ]:
from src.l3_m9_financial_compliance_risk import HumanInTheLoopWorkflow

# Initialize workflow
workflow = HumanInTheLoopWorkflow()
print("🔄 Initialized HITL Workflow\n")

# Step 1: Submit query
print("Step 1: Submit Query")
result = workflow.process_query(
    user_id="pm_001",
    query_text="Approve $8M increase in Tesla position based on Q3 earnings",
    transaction_amount=8_000_000,
    action_type="investment_decision",
    rag_response={"recommendation": "Increase position", "confidence": 0.85}
)

print(f"  Audit ID: {result['audit_id']}")
print(f"  Risk Level: {result['risk_level']}")
print(f"  Routed to: {result['routing']['reviewer_role']}")
print(f"  SLA: {result['routing']['sla_hours']} hours")
# Expected: HIGH risk, Senior Analyst, 4h SLA

print("\n" + "-"*60 + "\n")

# Step 2: Submit human review
print("Step 2: Submit Review Decision")
review_result = workflow.submit_review(
    audit_id=result['audit_id'],
    reviewer_id="analyst_senior_42",
    reviewer_role="Senior Analyst",
    decision_outcome=DecisionOutcome.REJECTED,
    supporting_evidence="Q3 earnings beat driven by $200M one-time tax benefit. "
                        "Excluding one-time gain, Tesla missed estimates. "
                        "Proposed trade breaches 5% concentration limit. "
                        "Recommend modified $4M increase to $16M (4% concentration)."
)

print(f"  Status: {review_result['status']}")
print(f"  Decision: {review_result['decision_outcome']}")
print(f"  Message: {review_result['message']}")
# Expected: Success, rejected, with reasoning

print("\n" + "-"*60 + "\n")

# Step 3: Retrieve and verify audit trail
print("Step 3: Verify Audit Trail")
audit_data = workflow.get_audit_trail()

print(f"  Total Entries: {audit_data['total_entries']}")
print(f"  Hash Chain Valid: {audit_data['hash_chain_valid']}")
print(f"  Validation Error: {audit_data['validation_error']}")
# Expected: 1 entry, valid hash chain

print("\n✅ SAVED_SECTION:7")

## Section 8: Real-World Scenario - Tesla $8M Position Increase

**Case Study from Augmented Script:**

**Background:**
- Current Tesla position: $12M (3% of AUM)
- Proposed increase: $8M → Total $20M (5% concentration)
- Q3 earnings beat expectations: $0.66 EPS vs $0.55 estimate

**Hidden Issue:**
- $200M one-time tax benefit inflated results
- Excluding one-time gain: $0.51 EPS (MISSED by $0.04)
- $20M position breaches 5% concentration limit (max 4.5%)

**Outcome:**
- **Cost:** 103 minutes analyst time (~$257)
- **Prevented Loss:** $500K-2M (concentration breach + earnings quality)
- **ROI:** 194,500% - 778,000%

**Expected:** SAVED_SECTION:8

In [ ]:
# Tesla Case Study Workflow
print("="*60)
print("REAL-WORLD CASE STUDY: Tesla $8M Position Increase")
print("="*60 + "\n")

tesla_workflow = HumanInTheLoopWorkflow()

# Stage 1: Portfolio Manager Query
print("Stage 1: Query Submission")
print("Portfolio Manager Request:")
print("  Current Position: $12M (3% of AUM)")
print("  Proposed Increase: $8M")
print("  Target Position: $20M (5% concentration)")
print("  Rationale: Q3 earnings beat expectations")

tesla_query = tesla_workflow.process_query(
    user_id="pm_portfolio_manager",
    query_text="Increase Tesla position from $12M to $20M based on Q3 earnings beat ($0.66 vs $0.55 estimate)",
    transaction_amount=8_000_000,
    action_type="investment_decision",
    rag_response={
        "recommendation": "Approve $8M increase",
        "confidence": 0.85,
        "reasoning": "Q3 earnings beat by 20%"
    }
)

print(f"\nSystem Classification:")
print(f"  Risk Level: {tesla_query['risk_level']}")
print(f"  Assigned to: {tesla_query['routing']['reviewer_role']}")
print(f"  SLA: {tesla_query['routing']['sla_hours']} hours")
# Expected: HIGH risk, Senior Analyst, 4h SLA

print("\n" + "-"*60 + "\n")

# Stage 2: Senior Analyst Review
print("Stage 2: Human Review by Senior Analyst (103 minutes)\n")
print("Analyst Deep Dive:")
print("  ✓ Q3 earnings: $0.66 EPS vs $0.55 estimate (beat)")
print("  ❌ BUT: $200M one-time tax benefit in footnotes")
print("  ❌ Excluding one-time: $0.51 EPS (MISSED by $0.04)")
print("  ❌ Proposed $20M = 5% concentration (limit: 4.5%)")
print("\nAnalyst Decision: REJECT original request")
print("Counterproposal: $4M increase to $16M (4% concentration)")

tesla_review = tesla_workflow.submit_review(
    audit_id=tesla_query['audit_id'],
    reviewer_id="analyst_senior_42",
    reviewer_role="Senior Analyst",
    decision_outcome=DecisionOutcome.MODIFIED,
    supporting_evidence=(
        "REJECTED original $8M increase. DETAILED ANALYSIS:\n"
        "1. Q3 Earnings Quality: $0.66 EPS headline vs $0.55 estimate appears strong.\n"
        "2. CRITICAL FINDING: 10-Q footnote 7 shows $200M one-time tax benefit.\n"
        "3. Adjusted EPS: $0.51 (MISSED estimate by $0.04, -7%).\n"
        "4. Concentration Risk: Proposed $20M position = 5% of AUM.\n"
        "5. Policy Breach: Internal limit is 4.5% per position.\n\n"
        "RECOMMENDATION: MODIFIED increase of $4M to $16M total (4% concentration).\n"
        "This maintains exposure while respecting risk controls and earnings quality concerns."
    )
)

print(f"\nReview Result: {tesla_review['decision_outcome'].upper()}")

print("\n" + "-"*60 + "\n")

# Stage 3: ROI Analysis
print("Stage 3: Cost-Benefit Analysis\n")
print("Cost:")
print("  Analyst Time: 103 minutes")
print("  Hourly Rate: $150/hour")
print("  Total Cost: $257")
print("\nPrevented Losses:")
print("  Concentration Breach: $500K-1M potential loss")
print("  Earnings Quality Issue: $1M-2M potential loss")
print("  Total Prevented: $500K-2M")
print("\nROI Calculation:")
print("  Conservative: ($500K - $257) / $257 = 194,500%")
print("  Aggressive: ($2M - $257) / $257 = 778,000%")
print("\n💡 Human review prevented catastrophic error!")

print("\n✅ SAVED_SECTION:8")

## Section 9: Failure Analysis & Debugging

**Common Failures from Augmented Script:**

1. **Missing Accounting Footnotes:** RAG misses one-time gains in earnings
2. **Domain Expertise Mismatch:** Credit analyst reviewing equity derivatives
3. **Approval Bottlenecks:** Queue backlogs missing market windows
4. **Fail-Unsafe Defaults:** System errors auto-approving high-stakes trades
5. **Risk Classifier Downtime:** Service crashes routing $50M as LOW risk

**Expected:** SAVED_SECTION:9

In [ ]:
print("="*60)
print("FAILURE ANALYSIS & DEBUGGING")
print("="*60 + "\n")

# Failure 1: Missing Accounting Footnotes
print("Failure 1: Missing Accounting Footnotes")
print("  Symptom: RAG recommends trade based on headline earnings")
print("  Root Cause: Retrieval doesn't include 10-K/10-Q footnotes")
print("  Fix: Enhance RAG to retrieve full footnote sections")
print("       Train reviewers to check 'Non-GAAP Reconciliation'")

print("\n" + "-"*60 + "\n")

# Failure 2: Domain Expertise Mismatch
print("Failure 2: Domain Expertise Mismatch")
print("  Symptom: Credit analyst assigned equity options trade")
print("  Root Cause: Routing based only on role, not skill matrix")
print("  Fix: Implement skill-based routing in addition to role")
print("       Maintain reviewer skill matrix in database")

print("\n" + "-"*60 + "\n")

# Failure 3: Approval Bottlenecks
print("Failure 3: Approval Bottlenecks")
print("  Symptom: Senior analyst queue has 47 pending reviews")
print("  Root Cause: No queue depth monitoring or overflow routing")
print("  Fix: Monitor queue depth, auto-escalate when >20 items")
print("       Implement backup reviewer routing")

# Demonstrate queue monitoring
queue_workflow = HumanInTheLoopWorkflow()
for i in range(5):
    queue_workflow.process_query(
        user_id=f"user_{i}",
        query_text=f"Query {i}",
        transaction_amount=2_000_000,
        action_type="investment_decision"
    )

audit_data = queue_workflow.get_audit_trail()
pending_count = len([e for e in audit_data['audit_trail'] if not e.get('reviewer_id')])
print(f"  Current Queue Depth: {pending_count} pending reviews")
# Expected: 5 pending reviews

print("\n" + "-"*60 + "\n")

# Failure 4: Fail-Unsafe Defaults
print("Failure 4: Fail-Unsafe Defaults")
print("  Symptom: Database timeout → system auto-approves $12M trade")
print("  Root Cause: Error handling defaults to approval (REGULATORY VIOLATION)")
print("  Fix: Implement fail-safe: on error, BLOCK execution")
print("       Send critical Slack alert, require CFO override")

print("\n" + "-"*60 + "\n")

# Failure 5: Risk Classifier Downtime
print("Failure 5: Risk Classifier Downtime")
print("  Symptom: Service crashes, $50M trade classified as LOW risk")
print("  Root Cause: No health check or fallback logic")
print("  Fix: Add service health checks before classification")
print("       Fallback to CRITICAL classification on failure")
print("       Implement circuit breaker pattern")

# Demonstrate fail-safe classification
print("\nFail-Safe Example:")
try:
    # Simulate service failure by passing extreme values
    risk_failsafe, reason_failsafe = classify_risk(
        query_text="Large transaction during service degradation",
        transaction_amount=50_000_000,
        action_type="investment_decision"
    )
    print(f"  Classification: {risk_failsafe.value}")
    print(f"  Reason: {reason_failsafe}")
    # Expected: CRITICAL (>$10M threshold)
except Exception as e:
    print(f"  ⚠️ Service failure detected: {e}")
    print(f"  Fail-safe: Escalating to CRITICAL risk")

print("\n✅ SAVED_SECTION:9")

## Section 10: Summary & Next Steps

**Key Takeaways:**

1. **Risk-Proportional Review:** Match scrutiny to financial impact (CRITICAL/HIGH/MEDIUM/LOW)
2. **Expertise-Matched Routing:** Assign to appropriate specialists with SLA tracking
3. **Audit-Ready by Default:** Tamper-proof hash chains for 7-year SOX compliance
4. **Fail-Safe Defaults:** Block on uncertainty, never auto-approve critical decisions
5. **Real ROI:** Tesla case study shows 194,500%-778,000% ROI preventing catastrophic errors

**Five HITL Design Principles:**
1. Risk-proportional review
2. Expertise-matched routing
3. Time-aware workflows
4. Audit-ready by default
5. Fail-safe defaults

**Production Deployment Checklist:**
- ✅ PostgreSQL for persistent audit trail storage
- ✅ Redis for priority queue management
- ✅ SendGrid/Slack notifications for reviewers
- ✅ Prometheus/Grafana for SLA monitoring
- ✅ Skill-based routing beyond role hierarchy
- ✅ Queue depth monitoring and overflow routing
- ✅ Circuit breaker patterns for service failures

**Next Steps:**
- Finance AI M9.5: Real-time Monitoring & Alerts
- Finance AI M10: Advanced Compliance Automation

**Expected:** SAVED_SECTION:10

In [ ]:
print("="*60)
print("MODULE COMPLETE: L3 M9.4 Human-in-the-Loop")
print("="*60 + "\n")

print("✅ You have successfully learned:")
print("   1. Risk-based query classification")
print("   2. Expertise-matched routing with SLA tracking")
print("   3. Tamper-proof audit trails with hash chains")
print("   4. Role-based approval workflows")
print("   5. Time-aware SLA compliance monitoring")
print("   6. Escalation hierarchies and fail-safe defaults")
print("   7. Real-world application (Tesla case study)")
print("   8. Common failure modes and debugging")

print("\n" + "-"*60 + "\n")

print("🎯 Key Metrics:")
print("   Cost per review: ~$257 (103 min analyst time)")
print("   ROI range: 194,500% - 778,000%")
print("   Prevented loss: $500K-2M per caught error")

print("\n" + "-"*60 + "\n")

print("📚 Next Steps:")
print("   1. Deploy API: uvicorn app:app --reload")
print("   2. Run tests: pytest tests/")
print("   3. Review README.md for production deployment")
print("   4. Explore Finance AI M9.5: Real-time Monitoring")

print("\n" + "="*60)
print("Thank you for completing L3 M9.4!")
print("="*60)

print("\n✅ SAVED_SECTION:10")